# **Computer vision** is the pratice of writing algorithms which can discover patterns in visual data. Such as the camera of a self-driving car recognizing the in front etc.,

#Get the data

The images we're working with are from the Food101 dataset (101 different classes of food): https://www.kaggle.com/dansbecker/food-101

But we are using only the modified part which only contains 2 classes that is (pizza and steak) using the image data modification in the link:
https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/extras/image_data_modification.ipynb

In [ ]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip

#unzip the downloaded file
zip_ref = zipfile.ZipFile("pizza_steak.zip")
zip_ref.extractall()
zip_ref.close()

# Inspect the data

Visualizing many samples of data


In [ ]:
!ls pizza_steak


In [ ]:
!ls pizza_steak/train/

In [ ]:
!ls pizza_steak/train/steak

In [ ]:
import os

#walk through pizza_steak directory and list number of files

for dirpath, dirnames, filenames in os.walk("pizza_steak"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")


So, there are 750 images of steak in trainset of steak, 250 images of steak images in testset of steak similarly the train and test sets of pizza's have 750 and 250 images respectfully.

In [ ]:
!ls -la pizza_steak

In [ ]:
#Another way to find out how many images are in a file
num_steak_images_train = len(os.listdir("pizza_steak/train/steak"))

num_steak_images_train

To visualize our images, first let's class names programmatically.

In [ ]:
import pathlib
import numpy as np
data_dir = pathlib.Path("pizza_steak/train")

class_names = np.array(sorted([item.name for item in data_dir.glob("*")])) #created a list of class names from the subdirectories

In [ ]:
class_names
# if we have a .DS_store or any other extra files or folders in your current folder you are accessing use if its your first file.
#class_names = class_names[1:]

In [ ]:
#Let's visualize our images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

def view_random_image(target_dir, target_class):
  #setting up the target directory
  target_folder = target_dir  +target_class

  #get a random image path
  random_image = random.sample(os.listdir(target_folder),1) #It randomly samples one of the image in the target folder.
  print(random_image)
  print(random_image[0])

  #Read in the image and plot it using the matplotlib
  img = mpimg.imread(target_folder + "/" + random_image[0]) #[0] because it return back as list ['x_ooo.img'] so we index so it return back as the string 'x_ooo.img'
  plt.imshow(img)
  plt.title(target_class)
  plt.axis("off")

  print(f"Image shape: {img.shape}") #shape is (224,224)
  return img

In [ ]:
#view the random image from the training dataset.
img = view_random_image(target_dir = "pizza_steak/train/",
                        target_class = "steak")

In [ ]:
import tensorflow as tf
tf.constant(img)

In [ ]:
#view the image shape
img.shape #returns width ,height , colour channels


In [ ]:
#Get all the pixel values between 0 and 1
img/255.

#An end to end example

Let's build a convolutional neural network to find patterns in our images:

* Load our images
* Preprocess our images
* Build a CNN to find patterns in our images.
* Compile our CNN
* Fit the CNN to our training data



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Set the seed
tf.random.set_seed(42)

#process data(get all of the pixel values between 0 & 1 also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale = 1./255)
valid_datagen = ImageDataGenerator(rescale = 1./255)

#setup the paths to our data directories
train_dir = "/content/pizza_steak/train/"
test_dir = "/content/pizza_steak/test/"

#Import data from  directories and turn it into batches
train_data = train_datagen.flow_from_directory(directory = train_dir,
                                               batch_size = 32,
                                               target_size = (224,224),
                                               class_mode = "binary",
                                               seed = 42
                                               )
valid_data = train_datagen.flow_from_directory(directory = test_dir,
                                               batch_size = 32,
                                               target_size = (224,224),
                                               class_mode = "binary",
                                               seed = 42
                                               )

#Build a CNN model (same as the tiny VGG on the CNN)

model_1 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters = 10,
                           kernel_size = 3,
                           activation ='relu',
                           input_shape = (224,224,3)),
    tf.keras.layers.Conv2D(10,3,activation = 'relu'),
    #tf.keras.layers.Activations(tf.nn.relu), we can use the activation this way instead of putting it into a Conv2D layer as in the above line.
    tf.keras.layers.MaxPool2D(pool_size=2,
                              padding = "valid"),
    tf.keras.layers.Conv2D(10,3,activation = 'relu'),
    tf.keras.layers.Conv2D(10,3,activation = 'relu'),
    tf.keras.layers.MaxPool2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1,activation = 'sigmoid')
])

#compile our CNN

model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])


In [ ]:
len(train_data)

In [ ]:
1500/32

# So, we have 47 batches of 32 images and labels
# we split it into batches because if try to fit all the 1500 images the computer might run out of memory

In [ ]:
train_data


In [ ]:
train_data.class_indices

In [ ]:
# Get a batch of data
images, labels = next(train_data)

# Display shapes of the images and labels
print(f"Images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")

# Example: Display the first image and its label
import matplotlib.pyplot as plt

plt.imshow(images[0])  # Display the first image
plt.title(f"Label: {labels[0]}")  # Show the label
plt.axis("off")
plt.show()


In [ ]:
import pandas as pd
import numpy as np

# Get a single batch of data
images, labels = next(train_data)

# Flatten each image (convert from 3D to 1D)
flattened_images = images.reshape(images.shape[0], -1)

# Create a DataFrame
df = pd.DataFrame(flattened_images)
df['Label'] = labels  # Add labels as a column

print(df.head())  # View the DataFrame


#Using the same model as before(classification model)

In [ ]:
#set random seed
tf.random.set_seed(42)

# Create a model to replicate the Tensorflow playground model

model_2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (224,224,3)),
    tf.keras.layers.Dense(4,activation = 'relu'),
    tf.keras.layers.Dense(4,activation = 'relu'),
    tf.keras.layers.Dense(1,activation = 'sigmoid')
])

# Compile the model

model_2.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

# Recreate train_data and valid_data here as they were defined earlier in the notebook
# to avoid issues with the data loader
#process data(get all of the pixel values between 0 & 1 also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale = 1./255)
valid_datagen = ImageDataGenerator(rescale = 1./255)

#setup the paths to our data directories
train_dir = "/content/pizza_steak/train/"
test_dir = "/content/pizza_steak/test/"

#Import data from  directories and turn it into batches
train_data = train_datagen.flow_from_directory(directory = train_dir,
                                               batch_size = 32,
                                               target_size = (224,224),
                                               class_mode = "binary",
                                               seed = 42
                                               )
valid_data = train_datagen.flow_from_directory(directory = test_dir,
                                               batch_size = 32,
                                               target_size = (224,224),
                                               class_mode = "binary",
                                               seed = 42
                                               )

# fit the model
history_2 = model_2.fit(train_data,
                        epochs =5,
                        steps_per_epoch = len(train_data)-1,
                        validation_data = valid_data,
                        validation_steps = len(valid_data)-1
                        )

Despite having more parameters model_2 underperforms over CNN(model_1)

In [ ]:
model_2.summary()

The trainable parameters are like patterns a model can learn from data. Intutively(more is better) normal classification just classifies based on the series of parametrs in its dense layer but convolutional layer seeks to sort out and learn most important patterns in an image. So even though it has less learnable parameters it has accuracy compared to normal classification.

model_1.summary()

# Binary classification

1. visualize the data
2. preprocess the data(scaling / normalizing)
3. Create a model(start with a baseline)
4. Fit the model
5. Evaluate the model
6. Adjust the different parameters and improve the model.
7. twerk with the model to get better results

# 1. Visualize the data


In [ ]:
plt.figure()
plt.subplot(1,2,1)
steak_img = view_random_image("pizza_steak/train/", "steak")

plt.subplot(1,2,2)
steak_img = view_random_image("pizza_steak/train/", "pizza")


# 2. Preprocess the data(prepare it for a model)

In [ ]:
#Define directory dataset paths

train_dir = "pizza_steak/train/"
test_dir = "pizza_steak/test/"

next step is to turn the data into
**batches**

A batch is a small subset of data. Rather than look at all -10000 images at one time, a model might only look at 32 at a time.

It does this for a couple of reasons:
1. 10,000 image(or more) might not fit into the memory of the processor(GPU).

2. Trying to learn patterns in 10,000 images in one hit could result in the model not being able to learn patterns very well.

3. Training batch size of more than 32 can result in test errors.

In [ ]:
!nvidia-smi

In [ ]:
# Create train and test data generators and rescale the data.

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1/255.)
test_datagen = ImageDataGenerator(rescale = 1/255.)

In [ ]:
#Load in our image data from the directories and turn them into batches
train_data = train_datagen.flow_from_directory(directory = train_dir, #Target directory of images
                                               target_size =(224,224),#Target size of images(height,width)
                                               class_mode = "binary",#type of data you're working with what type classification it could be.
                                               batch_size = 32 # size of minibatches to load data into
                                               )
test_data = train_datagen.flow_from_directory(directory = test_dir,
                                               target_size =(224,224),
                                               class_mode = "binary",
                                               batch_size = 32
                                               )

In [ ]:
#Get a sample of a train data batch
images , labels = next(train_data)
len(images),len(labels)


In [ ]:
images.shape,labels.shape

In [ ]:
#How many batches are there?
len(train_data)


In [ ]:
1500/32

In [ ]:
#Get the first two images
images[:2],images[0].shape

In [ ]:
images[7]

In [ ]:
images[:2].shape

In [ ]:
#view the first batch of labels
labels

### 3.Create a CNN model

It is a relatively simple model or existing result (try to beat the baseling through the experimentation)

In [ ]:
# Make the creating of our model easier
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPool2D,Activation
from tensorflow.keras import Sequential


In [ ]:
# Create the model
model_4 = Sequential([
    Conv2D(filters =10,
           kernel_size =3, #same as (3,3) it is the size of the sliding window going across the image
           strides = 1, # the size of steps the sliding window takes across the iage
           padding = "valid",# If 'same
           activation = "relu",
           input_shape = (224,224,3)), # input layer(specify input shape)
    Conv2D(10,3,activation = "relu"),
    Conv2D(10,3,activation = "relu"),
    #MaxPool2D(),
    Flatten(),
    Dense(1,activation = "sigmoid") #output layer (working with binary classificatio so only 1 output neuron)
])

In [ ]:
#compile the model

model_4.compile(loss = 'binary_crossentropy',
                optimizer = Adam(),
                metrics = ['accuracy'])

In [ ]:
model_4.summary()

# 4. Fit the model

In [ ]:
#check the lengths of training and test data generators
len(train_data),len(test_data)

In [ ]:
#fit the model

history_4 = model_4.fit(train_data, # this is the combination of labels(y) and sample data(x)
epochs = 5,
                        steps_per_epoch = len(train_data)-1,
                        validation_data = test_data,
                        validation_steps = len(test_data)-1,

                        )

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
#model_spl

# Model Definition
model_v = Sequential([
    Conv2D(10, 3, activation="relu", input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(10, 3, activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(1, activation="sigmoid")
])

# Compile Model
model_v.compile(
    loss="binary_crossentropy",
    optimizer=Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

# Learning Rate Scheduler
def scheduler(epoch, lr):
    return lr * 0.9 if epoch > 2 else lr

lr_scheduler = LearningRateScheduler(scheduler)

# Train Model
history_v = model_v.fit(
    train_data,
    epochs=5,
    steps_per_epoch=len(train_data)-1,
    validation_data=test_data,
    validation_steps=len(test_data)-1,
    callbacks=[lr_scheduler]
)


In [ ]:
import numpy as np
dummy_images = np.random.rand(10, 224, 224, 3).astype(np.float32)
dummy_labels = np.random.randint(0, 2, size=(10,)).astype(np.float32)

result = model_4.train_on_batch(dummy_images, dummy_labels)
print("Dummy data test result:", result)


In [ ]:
model_1.evaluate(test_data)

In [ ]:
model_1.summary()

# 5. Evaluating our model

In [ ]:
history_4.history

In [ ]:
import pandas as pd
pd.DataFrame(history_4.history).plot(figsize=(10,7))

In [ ]:
#plot the validation and training curves separately

def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend()

In [ ]:
#check out the loss and accuracy of model_4
plot_loss_curves(history_4)

If a model's **validation loss** starts to increase, it's likely that the model is **Overfitting** the training dataset(It learning patterns in the training dataset too well thus diminishing the model's ability to generalize to unseen data).

# 6. Adjust the model parameters

Fitting a machine learning model comes in 3 steps;

1. create a baseline
2. beat the baseline by overfitting a larger model
3. reduce overfitting

ways to induce overfitting

* Increase the number of conv layers
* Increase the number of conv filters
* Add another dense layer to the output of our flattened layer

Reduce Overfitting:

* Add data augmentation
* Add regularization layers(maxpool2d etc,)
* Add more data...

In [ ]:
# Create the model (this )
model_5 = tf.keras.Sequential([
    Conv2D(10,3,activation = "relu",input_shape= (224,224,3)),
    MaxPool2D(pool_size = 2),
    Conv2D(10,3,activation = "relu"),
    MaxPool2D(),
    Conv2D(10,3,activation = "relu"),
    MaxPool2D(),
    Flatten(),
    Dense(1,activation = "sigmoid")
])

In [ ]:
#compile the model

model_5.compile(loss = 'binary_crossentropy',
                optimizer = Adam(),
                metrics = ['accuracy'])

In [ ]:
# fit the model
history_5 = model_5.fit(train_data,
                epochs = 5,
                steps_per_epoch = len(train_data)-1,
                validation_data = test_data,
                validation_steps = len(valid_data)-1)

In [ ]:
model_5.summary()

In [ ]:
model_4.summary()

In [ ]:
#plot loss curves
plot_loss_curves(history_5)

# Data Augmentation.

It is the process of altering our training data, leading it ot have more diversity and in turn allowing our models to learn more generalizable patterns. Altering might mean adjusting the rotaintion of an image, flipping it, cropping it or something similar.


In [ ]:
#create ImageGenerator training instance with
train_datagen_augmented = ImageDataGenerator(rescale = 1/255.,
                                             rotation_range = 0.2, #how much do you rotate an image.
                                             shear_range = 0.2, # how much do you want to shear an image.
                                             zoom_range = 0.2, # zoom in randomly on an image

                                             width_shift_range = 0.2, #moves image around on the x-axis
                                             height_shift_range = 0.3, #moves image aroung on the y-axis
                                             horizontal_flip = True) #flips the image.
# Create ImageDataGenerator without data augmentation
train_datagen = ImageDataGenerator(rescale = 1/255.)

# Create ImageGenerator without data augmentation for the test dataset
test_datagen = ImageDataGenerator(rescale = 1/255.)

In [ ]:
#Import data and augment it from training data.
print("Augmented training images")
train_data_augmented = train_datagen_augmented.flow_from_directory(train_dir,
                                                                   target_size = (224,224),
                                                                   batch_size = 32,
                                                                   class_mode = "binary",
                                                                   shuffle = False
                                                                  )

#create non-augmented train data batches
print("Non-augmented training images")
test_data = train_datagen.flow_from_directory(train_dir,
                                              target_size = (224,224),
                                              batch_size =32,
                                              class_mode = "binary",
                                              shuffle = False)
#Create non-augmented test data batches
print("Non-augmented test images")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size = (224,224),
                                             batch_size = 32,
                                             class_mode = "binary",
                                             shuffle = False)


Data augmentation is only performed on the training data.

using the **ImageDataGenerator** the built-in data augmentation parameters our images are left as they are in the directories but are modified as they're loaded into the model.


# Visualizing the Augmented data

In [ ]:
import matplotlib.pyplot as plt
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [ ]:
# Create ImageDataGenerator for data augmentation
train_datagen_augmented = ImageDataGenerator(
    rescale=1/255.,
    rotation_range=0.2,  # Rotate the image
    shear_range=0.2,  # Shear the image
    zoom_range=0.2,  # Zoom in/out on the image
    width_shift_range=0.2,  # Shift the image horizontally
    height_shift_range=0.3,  # Shift the image vertically
    horizontal_flip=True  # Flip the image horizontally
)

# Create ImageDataGenerator for non-augmented data (training and testing)
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

In [ ]:


# Import data and augment it from the training data
print("Augmented training images:")
train_data_augmented = train_datagen_augmented.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    shuffle=False  # Ensure the same order of images for consistency
)

# Create non-augmented training data batches
print("Non-augmented training images:")
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    shuffle=False  # Ensure the same order of images for consistency
)

# Create non-augmented testing data batches
print("Non-augmented test images:")
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    shuffle=False
)


In [ ]:

# Fetch a batch of original and augmented images
original_images, labels = next(train_data)  # Non-augmented images
augmented_images, augmented_labels = next(train_data_augmented)  # Augmented images

# Ensure batch size consistency
assert len(original_images) == len(augmented_images), "Mismatch in batch sizes!"

# Randomly select an image from the batch
random_number = random.randint(0, len(original_images) - 1)
print(f"Showing image number: {random_number}")

# Display the original image
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(original_images[random_number])
plt.title("Original Image")
plt.axis(False)

# Display the corresponding augmented image
plt.subplot(1, 2, 2)
plt.imshow(augmented_images[random_number])
plt.title("Augmented Image")
plt.axis(False)

plt.tight_layout()
plt.show()

Creating the model with augmented data


In [ ]:
# Create a model(same as model_5)
model_6 = Sequential([
    Conv2D(10,3,activation = "relu"),
    MaxPool2D(pool_size =2),
    Conv2D(10,3,activation = "relu"),
    MaxPool2D(),
    Conv2D(10,3,activation = "relu"),
    MaxPool2D(),
    Flatten(),
    Dense(1,activation="sigmoid")

])

#compile the moedl

model_6.compile(loss = "binary_crossentropy",
                optimizer = Adam(),
                metrics = ["accuracy"])

# Fit the model
history_6 = model_6.fit(train_data_augmented,epochs=5,
                        steps_per_epoch = len(train_data_augmented)-1,
                        validation_data = test_data,
                        validation_steps = len(test_data)-1)

# It takes more time as it the data flows in the data gets augmented which takes more time.


In [ ]:
#Check our models training curves
history_6.history
#plot_loss_curves(history_6)

In [ ]:
plot_loss_curves(history_6)

Let's shuffle our augmented training data and train another model same as before.

In [ ]:
#Import data and augment it and then shuffle from training directory.
train_data_augmented_shuffled = train_datagen_augmented.flow_from_directory(train_dir,
                                                                            target_size = (224,224),
                                                                            class_mode= "binary",
                                                                            batch_size =32,
                                                                            shuffle = True)

In [ ]:
#Create the model.
# Create a model(same as model_5)
model_7 = Sequential([
    Conv2D(10,3,activation = "relu",input_shape = (224,224,3)),
    MaxPool2D(pool_size =2),
    Conv2D(10,3,activation = "relu"),
    MaxPool2D(),
    Conv2D(10,3,activation = "relu"),
    MaxPool2D(),
    Flatten(),
    Dense(1,activation="sigmoid")

])

#compile the moedl

model_7.compile(loss = "binary_crossentropy",
                optimizer = Adam(),
                metrics = ["accuracy"])

# Fit the model
history_7 = model_7.fit(train_data_augmented_shuffled,epochs=5,
                        steps_per_epoch = len(train_data_augmented_shuffled)-1,
                        validation_data = test_data,
                        validation_steps = len(test_data)-1)

# It takes more time as it the data flows in the data gets augmented which takes more time.



In [ ]:
#Plot loss curves
plot_loss_curves(history_7)

When shuffleing the training data, the model gets exposed to all different kinds of data during training, thus enabling it learn features across wide array of images.


* Increase the number of model layers(e.g add more 'Conv2D'/ 'MaxPool2D' layers)   

* Increase the number of filters in each convolutional layer

* Train for longer (more epochs)

* Find an ideal learning rate

* Get more data

* Use **transfer learning** to leverage what another image model has learn and adjust it for our own use case.

# Make a prediction with our trained model on our own custom data.

In [ ]:
print(class_names)

In [ ]:
!wget https://www.istockphoto.com/photo/grilling-steaks-on-flaming-grill-and-shot-with-selective-focus-gm594465522-101933167

In [ ]:
#view our example image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
# import a file from your desktop named 03-steak.jpeg and continue with below code for visualizing.
steak = mpimg.imread("steak_img.jpg")
plt.imshow(steak)
plt.axis(False)

In [ ]:
steak.shape

when you train a neural network and you want make a prediction with it on you own custom data, it's important that the custom data is preprocessed into the same format as the data the model was trained on.

In [ ]:
# Create a function to import an image and resize it to be able to be used with the model.
def load_and_prep_image(filename, img_shape=224):
  """
  Reads an image from filename, turns it into a tensor and reshapes it to
  (img_shape, img_shape, colour_channel).
  """
  # read in the image
  img = tf.io.read_file(filename)
  # Decode the read file into a tensor & ensure 3 colour channels
  # (our model is trained on images with 3 colour channels

  img = tf.image.decode_image(img)

  #resize the image
  img = tf.image.resize(img,size = [img_shape,img_shape])

  #rescale the image (get all values between 0 and 1)
  img = img/255.
  return img

In [ ]:
# Load in and preprocess our custom image
steak = load_and_prep_image("steak_img.jpg")

In [ ]:
# This expands the dimension to pass in the batch size of the input as the model expects it.
expanded_steak = tf.expand_dims(steak,axis = 0)
expanded_steak.shape

In [ ]:
pred = model_7.predict(expanded_steak)

In [ ]:
# we can index the predicted class by rounding the prediction probability
pred_class = class_names[int(tf.round(pred))]
pred_class


In [ ]:
def pred_and_plot(model,filename,class_names=class_names):
  """

  imports an image located at the file name, makes a prediction with model and plots
  the image with predicted class as the title. """

  # import the target image and preprocess it
  img = load_and_prep_image(filename)

  # add an extra dimension to the image
  img_expanded = tf.expand_dims(img,axis = 0)

  # make a prediction
  pred = model.predict(img_expanded)

  # Get the predicted class
  pred_class = class_names[int(tf.round(pred))]

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False)

In [ ]:
#Test  our model on a custom image
pred_and_plot(model_7,"steak_img.jpg")

In [ ]:
!wget https://ohsweetbasil.com/wp-content/uploads/how-to-make-authentic-margherita-pizza-at-home-recipe-4.jpg

In [ ]:
pred_and_plot(model_7,"xyz.jpg")

# Multi-class classification

1. Become one with the data
2. preprocess the data (get it ready for a model)
3. Create a model (start with a baseline)
4. Fit the model.(overfit to make sure it works)
5. Evaluate the model.
6. Adjust different hyperparameters and improve the model(try to beat baseline/ reduce overfitting)
7. Repeat until satisfied